In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os, sys

# Data

In [2]:
melanoma_df = pd.read_csv('melanoma_allFrames_w_death.csv')
melanoma_df.head()

,Unnamed: 0,timestamp,stay_id,anchor_age,patientweight,los,gender,alb,aniongap,bun,...,pnutrition,ponutrition,packedrbc,paralytics,sedation,tpnutrition,vasoactive,vasopressors,hours-p-iv,deceased
0,0,2160-10-17 19:00:00,30062923,79,147.0,1.027708,0,0.00,14.0,14.0,...,0,0,0,0,4,0,0,0,NaN,0
1,1,2124-04-08 03:00:00,30065290,75,69.6,2.841146,1,0.00,10.0,21.0,...,0,0,0,0,1,0,0,0,NaN,0
2,2,2122-04-22 04:00:00,30066446,83,82.2,6.355139,1,2.65,17.0,7.0,...,0,0,0,0,0,0,0,0,NaN,0
3,3,2158-06-30 18:00:00,30139247,83,75.0,2.832755,0,0.00,15.0,32.0,...,0,0,0,0,0,0,0,0,NaN,0
4,4,2158-07-01 00:00:00,30139247,83,75.0,2.832755,0,0.00,15.0,32.0,...,0,0,0,0,0,0,0,0,NaN,0


# RNN with LSTM layers

We will treat this as a text classification problem, where each vital is a word

### Classification

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

TRAIN_COLS = ['anchor_age', 'patientweight', 'los', 'gender',
       'alb', 'aniongap', 'bun', 'crp', 'ca', 'chloride', 'creatinine',
       'fibrinogen', 'glucose', 'hgb', 'k', 'mg', 'na', 'p', 'platelets',
       'troponin', 'wbc', 'apneainterval', 'artco2p', 'arto2p', 'expratio',
       'hr', 'inspratio', 'insptime', 'nibpd', 'nibpm', 'nibps', 'pip', 'rr',
       'spo2', 'temp', 'urine', 'vm', 'vt', 'betablockers', 'ca-iv',
       'ca-noniv', 'cablockers', 'dextrose', 'fluids', 'insulin', 'k-iv',
       'hours-k-iv', 'loopdiuretics', 'mg-iv', 'mg-noniv', 'hours-mg-noniv',
       'p-iv', 'p-noniv', 'pnutrition', 'ponutrition', 'packedrbc',
       'paralytics', 'sedation', 'tpnutrition', 'vasoactive', 'vasopressors',
       'hours-p-iv']

pred_var = 'deceased'

X = melanoma_df[TRAIN_COLS]
X = X.fillna(-1)
y = melanoma_df[pred_var]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
x_t, X_val, y_t, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0)


In [19]:
def get_compiled_model():

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(x_t.shape[1], activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=tf.keras.optimizers.Adam(1e-4),
                  metrics=['accuracy'])
    return model
model = get_compiled_model()

In [20]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_t, y_t))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

model.fit(train_dataset, 
          epochs=20,
          validation_data=val_dataset,
          validation_steps=30)

Epoch 1/20

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

15/15 [==============================] - 0s 7ms/step - loss: 0.5921 - accuracy: 0.9006 - val_loss: 0.4118 - val_accuracy: 0.9324
Epoch 2/20
15/15 [==============================] - 0s 2ms/step - loss: 0.4590 - accuracy: 0.9388
Epoch 3/20
15/15 [==============================] - 0s 2ms/step - loss: 0.4370 - accuracy: 0.9538
Epoch 4/20
15/15 [==============================] - 0s 2ms/step - loss: 0.4604 - accuracy: 0.9576
Epoch 5/20
15/15 [==============================] - 0s 2ms/step - loss: 0.

In [45]:
preds = model.predict(X_test)
preds = preds.flatten()

# Binarize preds
preds[preds <= 0] = 0
preds[preds > 0] = 1

correct_idx, = np.where(preds == y_test.values) 
incorrect_idx, = np.where(preds != y_test.values) 
correct = preds[correct_idx] 
incorrect = preds[incorrect_idx]
tp = correct[correct == 1] # True positives
fp = incorrect[incorrect == 1] # False positives

tn = correct[correct == 0] # True negatives
fn =  incorrect[incorrect == 0]# False negatives


print("Precision: ", len(tp)/(len(tp) + len(fp)))
print("Recall: ", len(tp)/(len(tp) + len(fn)))
print("Accuracy", len(correct) / (len(correct) + len(incorrect)))

Precision:  0.21621621621621623
Recall:  0.4444444444444444
Accuracy 0.9247104247104247


### Regression

In [107]:
from sklearn.ensemble import GradientBoostingRegressor

TRAIN_COLS = ['anchor_age', 'patientweight', 'gender',
       'alb', 'aniongap', 'bun', 'crp', 'ca', 'chloride', 'creatinine',
       'fibrinogen', 'glucose', 'hgb', 'k', 'mg', 'na', 'p', 'platelets',
       'troponin', 'wbc', 'apneainterval', 'artco2p', 'arto2p', 'expratio',
       'hr', 'inspratio', 'insptime', 'nibpd', 'nibpm', 'nibps', 'pip', 'rr',
       'spo2', 'temp', 'urine', 'vm', 'vt', 'betablockers', 'ca-iv',
       'ca-noniv', 'cablockers', 'dextrose', 'fluids', 'insulin', 'k-iv',
       'hours-k-iv', 'loopdiuretics', 'mg-iv', 'mg-noniv', 'hours-mg-noniv',
       'p-iv', 'p-noniv', 'pnutrition', 'ponutrition', 'packedrbc',
       'paralytics', 'sedation', 'tpnutrition', 'vasoactive', 'vasopressors',
       'hours-p-iv',  'deceased']

pred_var = 'los'

X = melanoma_df[TRAIN_COLS]
X = X.fillna(-1)
y = melanoma_df[pred_var]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
x_t, X_val, y_t, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

In [108]:
def get_compiled_model():

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(len(x_t.columns), activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1),
                  loss='mean_absolute_error')
    return model

model = get_compiled_model()

In [122]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_t, y_t))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

model.fit(train_dataset, 
          epochs=20,
          validation_data=val_dataset,
          validation_steps=30)


Epoch 1/20
15/15 [==============================] - 0s 2ms/step - loss: 0.6620 - accuracy: 0.9565 - val_loss: 0.6706 - val_accuracy: 0.9565
Epoch 2/20
15/15 [==============================] - 0s 2ms/step - loss: 0.6545 - accuracy: 0.9570
Epoch 3/20
15/15 [==============================] - 0s 2ms/step - loss: 0.6780 - accuracy: 0.9560
Epoch 4/20
15/15 [==============================] - 0s 2ms/step - loss: 0.6219 - accuracy: 0.9586
Epoch 5/20
15/15 [==============================] - 0s 2ms/step - loss: 0.5959 - accuracy: 0.9613
Epoch 6/20
15/15 [==============================] - 0s 2ms/step - loss: 0.6549 - accuracy: 0.9565
Epoch 7/20
15/15 [==============================] - 0s 2ms/step - loss: 0.6473 - accuracy: 0.9576
Epoch 8/20
15/15 [==============================] - 0s 2ms/step - loss: 0.6615 - accuracy: 0.9570
Epoch 9/20
15/15 [==============================] - 0s 2ms/step - loss: 0.6378 - accuracy: 0.9581
Epoch 10/20
15/15 [==============================] - 0s 2ms/step - loss: 0.6